In [ ]:
import numpy as np
import glob
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = '../data/'

In [ ]:
img_file = glob.glob(os.path.join(path, 'original_img/*'))
img_file[0]

In [ ]:
name = []
name.append('dummy.jpg')
name.append('joker.jpg')
for suite in ['s', 'h', 'd', 'c']:
    for num in range(1,14):
        name.append(suite+str(num)+'.jpg')
name.append('back.jpg')

In [ ]:
img = cv2.imread(img_file[12])
img = img[0:2000, :]
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(gray, cmap='gray')

In [ ]:
ret, thresh = cv2.threshold(gray, 80, 255, cv2.THRESH_TOZERO_INV)
ret, thresh = cv2.threshold(1-thresh, 80, 255, cv2.THRESH_TOZERO_INV)
plt.imshow(thresh, cmap='gray')

In [ ]:
thresh2 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 99, 20)
plt.imshow(thresh2, cmap='gray')

In [ ]:
thresh3 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
plt.imshow(thresh3, cmap='gray')

In [ ]:
contours = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[1]
th_area = img.shape[0] * img.shape[1] / 100
contours_large = list(filter(lambda c:cv2.contourArea(c) > th_area, contours))

In [ ]:
def getRectByPoints(points):
    points = list(map(lambda x: x[0], points))

    points = sorted(points, key=lambda x:x[1])
    top_points = sorted(points[:2], key=lambda x:x[0])
    bottom_points = sorted(points[2:4], key=lambda x:x[0])
    points = top_points + bottom_points

    left = min(points[0][0], points[2][0])
    right = max(points[1][0], points[3][0])
    top = min(points[0][1], points[1][1])
    bottom = max(points[2][1], points[3][1])
    return (top, bottom, left, right)

In [ ]:
outputs = []
rects = []
approxes = []
for (i,cnt) in enumerate(contours_large):
    arclen = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.1*arclen, True)
    if len(approx) < 4:
        continue
    approxes.append(approx)
    rect = getRectByPoints(approx)
    rects.append(rect)

In [ ]:
approxes

In [ ]:
rects

In [ ]:
test = np.copy(img)
cv2.rectangle(test, (rects[0][2], rects[0][0]), (rects[0][3], rects[0][1]), (255, 0,0), 22)
plt.imshow(test)